In [ ]:
# Imports


In [1]:
# Class definition for S-Puzzle
class S_Puzzle:
    def __init__(self, initial_state):
        # Assume input initial_state is valid
        self.__size = len(initial_state)
        self.set_state(initial_state)
    
    def get_size(self):
        return self.__size
    
    def get_state(self):
        return self.__state
    
    def set_state(self, state):
        # Assume input state is valid, and same size as self.__size
        self.__state = state
        
    def get_current_state_children(self):
        children = []
        n = self.__size
        state = self.__state
        
        # Horizontal swaps
        for row in range(n):
            row_tuple = self.__state[row]
            for col in range(n-1):
                new_row = row_tuple[:col] + (row_tuple[col+1],) + (row_tuple[col],) + row_tuple[col+2:]
                new_tuple = state[:row] + (new_row,) + state[row+1:]
                children.append(new_tuple)
        
        # Vertical swaps
        for row in range(n-1):
            row1 = state[row]
            row2 = state[row+1]
            for col in range(n):
                new_row1 = row1[:col] + (row2[col],) + row1[col+1:]
                new_row2 = row2[:col] + (row1[col],) + row2[col+1:]
                new_tuple = state[:row] + (new_row1,) + (new_row2,) + state[row+2:]
                children.append(new_tuple)
        
        return children
    
    def is_current_state_goal(self):
        n = self.__size
        prev = 0
        for row in range(n):
            for col in range(n):
                if self.__state[row][col] != prev + 1:
                    return False
                prev += 1
        return True

In [2]:
# DFS Solver class definition
class DFS_Solver:
    def solve(self, puzzle):
        self.__seen_set = set()
        self.__solution_stack = []
        self.__search_stack = []
        success = self.__solve_dfs(puzzle)
        
        return success, self.__solution_stack, self.__search_stack
        
    def __solve_dfs(self, puzzle):
        state = puzzle.get_state()
        
        self.__search_stack.append(state)
        self.__solution_stack.append(state)
        self.__seen_set.add(state)
        
        if puzzle.is_current_state_goal():
            return True
        
        children = puzzle.get_current_state_children()
        
        for child in children:
            if child in self.__seen_set:
                continue
            
            puzzle.set_state(child)
            success = self.__solve_dfs(puzzle)
            
            if success:
                return True
        
        self.__solution_stack.pop()
        return False

In [7]:
# DFS Solver class definition
class Iterative_Deepening_Solver:
    def solve(self, puzzle):
        initial_state = puzzle.get_state()
        global_search_stack = []
        limit = 0
        while True:
            self.__seen_set = set()
            self.__solution_stack = []
            self.__search_stack = []
            self.__max_depth_seen = 0
            puzzle.set_state(initial_state)
            success = self.__solve_dfs(puzzle, 0, limit)
            
            global_search_stack.append(self.__search_stack)
            
            if success or self.__max_depth_seen < limit:
                break
            
            limit += 1
        
        return success, self.__solution_stack, global_search_stack
        
    def __solve_dfs(self, puzzle, current_depth, limit):
        if current_depth > limit:
            return False
        
        if current_depth > self.__max_depth_seen:
            self.__max_depth_seen = current_depth
        
        state = puzzle.get_state()
        
        self.__search_stack.append(state)
        self.__solution_stack.append(state)
        self.__seen_set.add(state)
        
        if puzzle.is_current_state_goal():
            return True
        
        children = puzzle.get_current_state_children()
        
        for child in children:
            if child in self.__seen_set:
                continue
            
            puzzle.set_state(child)
            success = self.__solve_dfs(puzzle, current_depth + 1, limit)
            
            if success:
                return True
        
        self.__solution_stack.pop()
        return False

In [9]:
class A_Star_Solver:
    pass

In [ ]:
%%time

puzzle = S_Puzzle( ( (2,3), (4,1) ) )
dfs_solver = DFS_Solver()

# dfs_solver.solve(puzzle)

In [8]:
%%time

puzzle = S_Puzzle( ((6, 1, 2), (7, 8, 3), (5, 4, 9)) )
id_solver = Iterative_Deepening_Solver()

id_solver.solve(puzzle)

Wall time: 47.9 ms


(True,
 [((6, 1, 2), (7, 8, 3), (5, 4, 9)),
  ((1, 6, 2), (7, 8, 3), (5, 4, 9)),
  ((1, 2, 6), (7, 8, 3), (5, 4, 9)),
  ((1, 2, 6), (7, 8, 3), (4, 5, 9)),
  ((1, 2, 6), (7, 5, 3), (4, 8, 9)),
  ((1, 2, 3), (7, 5, 6), (4, 8, 9)),
  ((1, 2, 3), (4, 5, 6), (7, 8, 9))],
 [[((6, 1, 2), (7, 8, 3), (5, 4, 9))],
  [((6, 1, 2), (7, 8, 3), (5, 4, 9)),
   ((1, 6, 2), (7, 8, 3), (5, 4, 9)),
   ((6, 2, 1), (7, 8, 3), (5, 4, 9)),
   ((6, 1, 2), (8, 7, 3), (5, 4, 9)),
   ((6, 1, 2), (7, 3, 8), (5, 4, 9)),
   ((6, 1, 2), (7, 8, 3), (4, 5, 9)),
   ((6, 1, 2), (7, 8, 3), (5, 9, 4)),
   ((7, 1, 2), (6, 8, 3), (5, 4, 9)),
   ((6, 8, 2), (7, 1, 3), (5, 4, 9)),
   ((6, 1, 3), (7, 8, 2), (5, 4, 9)),
   ((6, 1, 2), (5, 8, 3), (7, 4, 9)),
   ((6, 1, 2), (7, 4, 3), (5, 8, 9)),
   ((6, 1, 2), (7, 8, 9), (5, 4, 3))],
  [((6, 1, 2), (7, 8, 3), (5, 4, 9)),
   ((1, 6, 2), (7, 8, 3), (5, 4, 9)),
   ((1, 2, 6), (7, 8, 3), (5, 4, 9)),
   ((1, 6, 2), (8, 7, 3), (5, 4, 9)),
   ((1, 6, 2), (7, 3, 8), (5, 4, 9)),
   ((1, 6